# LLM Optimization Modelling Experiment

In [661]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [674]:
problem = '''You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7'''

## 2. Generate the mathematical model

In [675]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [676]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model

**Sets:**

*  $T = \{1, 2,..., 10\}$: Set of time steps.
*  $N = \{A, B, C, D, E, F, G\}$: Set of nodes in the network.
*  $E = \{(A, B), (A, C), (A, F), (B, C), (B, D), (B, E), (C, E), (C, F), (D, E), (E, G), (F, B), (F, G)\}$: Set of directed edges representing pipes.

**Parameters:**

*  $c_{ij}$: Capacity of pipe $(i,j) \in E$.  
    *  $c_{AB} = 3, c_{AC} = 6, c_{AF} = 1, c_{BC} = 3, c_{BD} = 10, c_{BE} = 4, c_{CE} = 4, c_{CF} = 4, c_{DE} = 5, c_{EG} = 5, c_{FB} = 12, c_{FG} = 7$
*  $f(t) = \max(-4/30t^3 + t^2 - 0.234t + 3, 0)$: Inflow function at node A at time $t \in T$.

**Decision Variables:**

*  $x_{ij}(t)$: Flow of water through pipe $(i, j) \in E$ at time $t \in T$.
*  $y_{ij}$: Binary variable, equal to 1 if pipe $(i,j) \in E$ is activated, and 0 otherwise.
*  $r_i(t)$: Amount of water released at node $i \in N$ at time $t \in T$.

**Objective Function:**

Maximize the total outflow at node G over all time steps:

$\max \sum_{t \in T} x_{EG}(t)$

**Constraints:**

1. **Flow Conservation:** For each node $i \in N$ and time $t \in T$:
    *  $\sum_{j:(i,j) \in E} x_{ij}(t) + r_i(t) = \sum_{j:(j,i) \in E} x_{ji}(t-1) + f(t)$ if $i = A$ 
    *  $\sum_{j:(i,j) \in E} x_{ij}(t) + r_i(t) = \sum_{j:(j,i) \in E} x_{ji}(t-1)$ if $i \neq A$
2. **Capacity Constraints:** For each pipe $(i,j) \in E$ and time $t \in T$:
    *  $0 \leq x_{ij}(t) \leq c_{ij}y_{ij}$
3. **Pipe Activation Limit:**
    *  $\sum_{(i,j) \in E} y_{ij} = 10$
4. **No Storage at Nodes:** 
    *  $\sum_{j:(i,j) \in E} x_{ij}(t) + r_i(t) = \sum_{j:(j,i) \in E} x_{ji}(t+1)$ for all $i \in N$ and $t \in T \setminus \{10\}$
5. **Non-negativity:**
    *  $x_{ij}(t) \geq 0$ for all $(i,j) \in E$, $t \in T$
    *  $r_i(t) \geq 0$ for all $i \in N$, $t \in T$
6. **Binary Constraint:**
    *  $y_{ij} \in \{0, 1\}$ for all $(i,j) \in E$ 

This model defines the problem of maximizing water flow by choosing which pipes to activate and how much water to send through each pipe at each time step. The constraints ensure flow conservation, respect pipe capacities, limit the number of activated pipes, prevent water storage at nodes, and ensure non-negativity of flows.


## 3. Generate the pyomo code

In [677]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [678]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define sets
T = range(1, 11)  # Set of time steps
N = ['A', 'B', 'C', 'D', 'E', 'F', 'G']  # Set of nodes
E = {
    ('A', 'B'): 3, ('A', 'C'): 6, ('A', 'F'): 1,
    ('B', 'C'): 3, ('B', 'D'): 10, ('B', 'E'): 4,
    ('C', 'E'): 4, ('C', 'F'): 4, 
    ('D', 'E'): 5, 
    ('E', 'G'): 5, 
    ('F', 'B'): 12, ('F', 'G'): 7
}  # Set of edges (pipes) with capacities

# Define parameters
def f(t):
    return max(-4/30*t**3 + t**2 - 0.234*t + 3, 0)  # Inflow function at node A

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(E, T, within=pyo.NonNegativeReals)  # Flow through pipes
model.y = pyo.Var(E, within=pyo.Binary)  # Pipe activation
model.r = pyo.Var(N, T, within=pyo.NonNegativeReals)  # Water released at nodes

# Define objective function
model.obj = pyo.Objective(expr=sum(model.x['E', 'G', t] for t in T), sense=pyo.maximize)

# Define constraints
# 1. Flow conservation
model.flow_conservation = pyo.ConstraintList()
for i in N:
    for t in T:
        if i == 'A':
            model.flow_conservation.add(
                sum(model.x[i, j, t] for (i, j) in E if (i, j) in E) + model.r[i, t] == 
                sum(model.x[j, i, t-1] for (j, i) in E if (j, i) in E) + f(t) 
            )
        else:
            model.flow_conservation.add(
                sum(model.x[i, j, t] for (i, j) in E if (i, j) in E) + model.r[i, t] == 
                sum(model.x[j, i, t-1] for (j, i) in E if (j, i) in E) 
            )

# 2. Capacity constraints
model.capacity = pyo.ConstraintList()
for (i, j) in E:
    for t in T:
        model.capacity.add(model.x[i, j, t] <= E[(i, j)] * model.y[i, j])

# 3. Pipe activation limit
model.activation_limit = pyo.Constraint(expr=sum(model.y[i, j] for (i, j) in E) == 10)

# 4. No storage at nodes
model.no_storage = pyo.ConstraintList()
for i in N:
    for t in T[:-1]:  # Exclude last time step
        model.no_storage.add(
            sum(model.x[i, j, t] for (i, j) in E if (i, j) in E) + model.r[i, t] == 
            sum(model.x[j, i, t+1] for (j, i) in E if (j, i) in E)
        )

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if you have it installed
solver.solve(model)

# Print results
print("Total outflow at node G:", pyo.value(model.obj))

# You can access the optimal values of decision variables like this:
# print("Flow through pipe (A, B) at time 1:", pyo.value(model.x['A', 'B', 1]))
# print("Is pipe (A, C) activated?:", pyo.value(model.y['A', 'C']))
# print("Water released at node B at time 5:", pyo.value(model.r['B', 5]))
```

**Sample Data:**

*   The capacities of the pipes (`c_ij`) are defined directly in the `E` dictionary. 

This code defines the model, sets the objective, adds the constraints, solves it using the GLPK solver (you might need to install it), and then prints the results. You can uncomment the last lines to print specific values of the decision variables. 


## 4. Input problem data and try running the generated code

In [685]:
import pyomo.environ as pyo

# Define sets
T = range(1, 11)  # Set of time steps
N = ['A', 'B', 'C', 'D', 'E', 'F', 'G']  # Set of nodes
E = {
    ('A', 'B'): 3, ('A', 'C'): 6, ('A', 'F'): 1,
    ('B', 'C'): 3, ('B', 'D'): 10, ('B', 'E'): 4,
    ('C', 'E'): 4, ('C', 'F'): 4, 
    ('D', 'E'): 5, 
    ('E', 'G'): 5, 
    ('F', 'B'): 12, ('F', 'G'): 7
}  # Set of edges (pipes) with capacities

# Define parameters
def f(t):
    return max(-4/30*t**3 + t**2 - 0.234*t + 3, 0)  # Inflow function at node A

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(E, T, within=pyo.NonNegativeReals)  # Flow through pipes
model.y = pyo.Var(E, within=pyo.Binary)  # Pipe activation
model.r = pyo.Var(N, T, within=pyo.NonNegativeReals)  # Water released at nodes

# Define objective function
model.obj = pyo.Objective(expr=sum(model.x['E', 'G', t] for t in T), sense=pyo.maximize)

# Define constraints
# 1. Flow conservation
model.flow_conservation = pyo.ConstraintList()
for i in N:
    for t in T:
        if i == 'A':
            model.flow_conservation.add(
                sum(model.x[i, j, t] for (i, j) in E if (i, j) in E) + model.r[i, t] == 
                sum(model.x[j, i, t-1] for (j, i) in E if (j, i) in E) + f(t) 
            )
        else:
            model.flow_conservation.add(
                sum(model.x[i, j, t] for (i, j) in E if (i, j) in E) + model.r[i, t] == 
                sum(model.x[j, i, t-1] for (j, i) in E if (j, i) in E) 
            )

# 2. Capacity constraints
model.capacity = pyo.ConstraintList()
for (i, j) in E:
    for t in T:
        model.capacity.add(model.x[i, j, t] <= E[(i, j)] * model.y[i, j])

# 3. Pipe activation limit
model.activation_limit = pyo.Constraint(expr=sum(model.y[i, j] for (i, j) in E) == 10)

# 4. No storage at nodes
model.no_storage = pyo.ConstraintList()
for i in N:
    for t in T[:-1]:  # Exclude last time step
        model.no_storage.add(
            sum(model.x[i, j, t] for (i, j) in E if (i, j) in E) + model.r[i, t] == 
            sum(model.x[j, i, t+1] for (j, i) in E if (j, i) in E)
        )

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if you have it installed
solver.solve(model)

# Print results
print("Total outflow at node G:", pyo.value(model.obj))

# You can access the optimal values of decision variables like this:
# print("Flow through pipe (A, B) at time 1:", pyo.value(model.x['A', 'B', 1]))
# print("Is pipe (A, C) activated?:", pyo.value(model.y['A', 'C']))
# print("Water released at node B at time 5:", pyo.value(model.r['B', 5]))

KeyError: "Index '('A', 'B', 0)' is not valid for indexed component 'x'"

## 5. Correct the code to verify model viability (optional)

In [688]:
import pyomo.environ as pyo

# Define sets
T = range(1, 11)  # Set of time steps
N = ['A', 'B', 'C', 'D', 'E', 'F', 'G']  # Set of nodes
E = {
    ('A', 'B'): 3, ('A', 'C'): 6, ('A', 'F'): 1,
    ('B', 'C'): 3, ('B', 'D'): 10, ('B', 'E'): 4,
    ('C', 'E'): 4, ('C', 'F'): 4, 
    ('D', 'E'): 5, 
    ('E', 'G'): 5, 
    ('F', 'B'): 12, ('F', 'G'): 7
}  # Set of edges (pipes) with capacities

# Define parameters
def f(t):
    return max(-4/30*t**3 + t**2 - 0.234*t + 3, 0)  # Inflow function at node A

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(E, T, within=pyo.NonNegativeReals)  # Flow through pipes
model.y = pyo.Var(E, within=pyo.Binary)  # Pipe activation
model.r = pyo.Var(N, T, within=pyo.NonNegativeReals)  # Water released at nodes

# Define objective function
model.obj = pyo.Objective(expr=sum(model.x['E', 'G', t] for t in T), sense=pyo.maximize)

#Define constraints
#1. Flow conservation
# model.flow_conservation = pyo.ConstraintList()
# for i in N:
#     for t in T:
#         if i == 'A':
#             model.flow_conservation.add(
#                 sum(model.x[i, j, t] for (i, j) in E if (i, j) in E) + model.r[i, t] == 
#                  f(t) 
#             )
#         else:
#             model.flow_conservation.add(
#                 sum(model.x[i, j, t] for (i, j) in E if (i, j) in E) + model.r[i, t] == 
#                 sum(model.x[j, i, t-1] for (j, i) in E if (j, i) in E) 
#             )

# 2. Capacity constraints
model.capacity = pyo.ConstraintList()
for (i, j) in E:
    for t in T:
        model.capacity.add(model.x[i, j, t] <= E[(i, j)] * model.y[i, j])

# 3. Pipe activation limit
model.activation_limit = pyo.Constraint(expr=sum(model.y[i, j] for (i, j) in E) == 10)

# 4. No storage at nodes
model.no_storage = pyo.ConstraintList()
for i in N:
    for t in T[:-1]:  # Exclude last time step
        model.no_storage.add(
            sum(model.x[i, j, t] for (i, j) in E if (i, j) in E) + model.r[i, t] == 
            sum(model.x[j, i, t+1] for (j, i) in E if (j, i) in E)
        )

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if you have it installed
solver.solve(model)

# Print results
print("Total outflow at node G:", pyo.value(model.obj))

# You can access the optimal values of decision variables like this:
# print("Flow through pipe (A, B) at time 1:", pyo.value(model.x['A', 'B', 1]))
# print("Is pipe (A, C) activated?:", pyo.value(model.y['A', 'C']))
# print("Water released at node B at time 5:", pyo.value(model.r['B', 5]))

Total outflow at node G: 50.0


In [689]:
model.pprint()

3 Var Declarations
    r : Size=70, Index={A, B, C, D, E, F, G}*{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
         ('A', 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
         ('A', 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
         ('A', 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
         ('A', 4) :     0 :   0.0 :  None : False : False : NonNegativeReals
         ('A', 5) :     0 :   0.0 :  None : False : False : NonNegativeReals
         ('A', 6) :     0 :   0.0 :  None : False : False : NonNegativeReals
         ('A', 7) :     0 :   0.0 :  None : False : False : NonNegativeReals
         ('A', 8) :     0 :   0.0 :  None : False : False : NonNegativeReals
         ('A', 9) :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('A', 10) :     0 :  None :  None : False :  True : NonNegativeReals
         ('B', 1) :     0 :   0.0 :  None : False : False : NonNega

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [690]:
print(response.text)

## Mathematical Optimization Model

**Sets:**

*  $T = \{1, 2,..., 10\}$: Set of time steps.
*  $N = \{A, B, C, D, E, F, G\}$: Set of nodes in the network.
*  $E = \{(A, B), (A, C), (A, F), (B, C), (B, D), (B, E), (C, E), (C, F), (D, E), (E, G), (F, B), (F, G)\}$: Set of directed edges representing pipes.

**Parameters:**

*  $c_{ij}$: Capacity of pipe $(i,j) \in E$.  
    *  $c_{AB} = 3, c_{AC} = 6, c_{AF} = 1, c_{BC} = 3, c_{BD} = 10, c_{BE} = 4, c_{CE} = 4, c_{CF} = 4, c_{DE} = 5, c_{EG} = 5, c_{FB} = 12, c_{FG} = 7$
*  $f(t) = \max(-4/30t^3 + t^2 - 0.234t + 3, 0)$: Inflow function at node A at time $t \in T$.

**Decision Variables:**

*  $x_{ij}(t)$: Flow of water through pipe $(i, j) \in E$ at time $t \in T$.
*  $y_{ij}$: Binary variable, equal to 1 if pipe $(i,j) \in E$ is activated, and 0 otherwise.
*  $r_i(t)$: Amount of water released at node $i \in N$ at time $t \in T$.

**Objective Function:**

Maximize the total outflow at node G over all time steps:

$\max \sum_{t \in T} x

In [691]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define sets
T = range(1, 11)  # Set of time steps
N = ['A', 'B', 'C', 'D', 'E', 'F', 'G']  # Set of nodes
E = {
    ('A', 'B'): 3, ('A', 'C'): 6, ('A', 'F'): 1,
    ('B', 'C'): 3, ('B', 'D'): 10, ('B', 'E'): 4,
    ('C', 'E'): 4, ('C', 'F'): 4, 
    ('D', 'E'): 5, 
    ('E', 'G'): 5, 
    ('F', 'B'): 12, ('F', 'G'): 7
}  # Set of edges (pipes) with capacities

# Define parameters
def f(t):
    return max(-4/30*t**3 + t**2 - 0.234*t + 3, 0)  # Inflow function at node A

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(E, T, within=pyo.NonNegativeReals)  # Flow through pipes
model.y = pyo.Var(E, within=pyo.Binary)  # Pipe activation
model.r = pyo.Var(N, T, within=pyo.NonNegativeReals)  # Water released at nodes

# Define objective function
model.obj = pyo.Objective(expr=sum(model.x['E', 'G', t] for t in T), sense=pyo.maximize)

# Define constraints
# 1. Flow conservation
model.flow_conservation = pyo.Constr